In [5]:
import quartz
import torch
import math
import dgl
import time
from icecream import ic

In [2]:
gate_set = ['h', 'cx', 't', 'tdg']
ecc_file = 'bfs_verified_simplified.json'
no_increase = False
include_nop = True
quartz_context = quartz.QuartzContext(
    gate_set=gate_set,
    filename=ecc_file,
    # TODO  we need to include xfers that lead to gate increase when training?
    # we may exclude them when generating the dataset for pre-training
    # TODO  to make the task easier, we exclude those xfers currently
    no_increase=no_increase,
    include_nop=include_nop,
)
num_xfers = quartz_context.num_xfers
parser = quartz.PyQASMParser(context=quartz_context)
ic(num_xfers)


ic| num_xfers: 3904


3904

In [3]:
qasm_str = 'OPENQASM 2.0;\ninclude "qelib1.inc";\nqreg q[5];\nh q[4];\ncx q[3],q[4];\ntdg q[4];\ncx q[2],q[4];\nt q[4];\ncx q[3],q[4];\ntdg q[4];\ncx q[3],q[2];\ntdg q[2];\ncx q[3],q[2];\nt q[3];\ncx q[2],q[4];\nh q[3];\nt q[4];\ncx q[1],q[3];\ntdg q[3];\ncx q[0],q[3];\nt q[3];\ncx q[1],q[3];\ntdg q[3];\ncx q[0],q[3];\ncx q[0],q[1];\nt q[3];\ntdg q[1];\nh q[3];\ncx q[0],q[1];\ncx q[3],q[4];\nt q[0];\nt q[1];\nt q[4];\ncx q[2],q[4];\ntdg q[4];\ncx q[3],q[4];\nt q[4];\ncx q[2],q[4];\ntdg q[4];\ncx q[2],q[3];\nh q[4];\nt q[3];\ncx q[2],q[3];\ntdg q[3];\nh q[3];\ncx q[1],q[3];\nt q[3];\ncx q[0],q[3];\ntdg q[3];\ncx q[1],q[3];\nt q[3];\ncx q[0],q[3];\ncx q[0],q[1];\ntdg q[3];\nt q[1];\nh q[3];\ncx q[0],q[1];\ntdg q[0];\ntdg q[1];\n'
s_time = time.time_ns()
dag = parser.load_qasm_str(qasm_str)
graph = quartz.PyGraph(context=quartz_context, dag=dag)
e_time = time.time_ns()
print(f'{(e_time - s_time) / 1e6} ms')
print(hash(graph))

3.902898 ms
-3680862461118220112


In [4]:
s_time = time.time_ns()
graph2 = quartz.PyGraph.from_qasm_str(context=quartz_context, qasm_str=qasm_str)
e_time = time.time_ns()
print(f'{(e_time - s_time) / 1e6} ms')
print(hash(graph2))

1.175393 ms
-3680862461118220112


In [6]:
while True:
    avxfers = []
    for i in range(graph.num_nodes):
        avxfers.append(
            graph.available_xfers(
                context=quartz_context,
                node=graph.get_node_from_id(id=i)
            )
        )
    avxfers2 = []
    for i in range(graph2.num_nodes):
        avxfers2.append(
            graph2.available_xfers(
                context=quartz_context,
                node=graph2.get_node_from_id(id=i)
            )
        )
    print(avxfers == avxfers2)

True
True
True
True
True


KeyboardInterrupt: 

In [ ]:
int(torch.multinomial(torch.Tensor([1,2,3,4]), 1))

In [ ]:
next_graph, next_nodes = \
    graph.apply_xfer_with_local_state_tracking(
        xfer=quartz_context.get_xfer_from_id(id=action.xfer),
        node=graph.get_node_from_id(id=action.node)
    )

In [ ]:
avxfers = []
for i in range(graph.num_nodes):
    avxfers.append(
        graph.available_xfers(
            context=quartz_context,
            node=graph.get_node_from_id(id=i)
        )
    )
avxfers

In [ ]:
qasm2 = graph.to_qasm_str()
g2 = quartz.PyGraph(context=quartz_context, dag=parser.load_qasm_str(qasm2))
avxfers2 = []
for i in range(g2.num_nodes):
    avxfers2.append(
        g2.available_xfers(
            context=quartz_context,
            node=g2.get_node_from_id(id=i)
        )
    )
avxfers2 == avxfers

In [13]:
N = 58
hr = 0.8
temp = 1 / (math.log(hr * (N - 1) / (1 - hr)))
ic(temp)

ic| temp: 0.18418425871932848


0.18418425871932848

In [40]:
nv = torch.rand(58)
# ic(nv)
nv[10] += 0.8
nv[20] += 1
print(nv)

tensor([2.3678e-01, 5.5466e-01, 8.6870e-01, 3.1310e-01, 3.9806e-01, 1.6154e-01,
        1.1253e-01, 2.5345e-01, 8.5483e-01, 8.7368e-01, 1.0489e+00, 9.9296e-01,
        2.2252e-01, 9.3534e-01, 4.4354e-01, 6.7339e-01, 8.9097e-01, 8.1088e-01,
        6.7483e-01, 8.5591e-01, 1.2983e+00, 5.4039e-01, 1.4504e-01, 8.5817e-01,
        7.3451e-01, 3.4302e-02, 3.7450e-01, 2.0502e-01, 5.2891e-01, 1.4889e-01,
        1.3780e-03, 6.5697e-01, 2.5934e-01, 2.3895e-01, 9.2069e-01, 8.2578e-01,
        9.1897e-02, 2.1039e-01, 1.2449e-03, 2.5067e-02, 1.6273e-01, 6.8860e-01,
        1.3174e-01, 3.7473e-01, 8.4814e-01, 8.6089e-01, 7.9354e-01, 4.7550e-01,
        8.8124e-02, 6.0274e-02, 3.2602e-01, 1.5500e-02, 6.6569e-01, 3.1290e-01,
        1.3094e-02, 5.1426e-01, 2.0129e-01, 8.0870e-01])


In [42]:
smnv = torch.softmax(nv, dim=0)
print(smnv, smnv[10], smnv[20])

tensor([0.0128, 0.0176, 0.0241, 0.0138, 0.0150, 0.0119, 0.0113, 0.0130, 0.0238,
        0.0242, 0.0288, 0.0273, 0.0126, 0.0257, 0.0157, 0.0198, 0.0246, 0.0227,
        0.0198, 0.0238, 0.0370, 0.0173, 0.0117, 0.0238, 0.0211, 0.0105, 0.0147,
        0.0124, 0.0171, 0.0117, 0.0101, 0.0195, 0.0131, 0.0128, 0.0254, 0.0231,
        0.0111, 0.0125, 0.0101, 0.0104, 0.0119, 0.0201, 0.0115, 0.0147, 0.0236,
        0.0239, 0.0223, 0.0163, 0.0110, 0.0107, 0.0140, 0.0103, 0.0197, 0.0138,
        0.0102, 0.0169, 0.0124, 0.0227]) tensor(0.0288) tensor(0.0370)


In [43]:
[torch.multinomial(smnv, 1) for _ in range(10)]

[tensor([52]),
 tensor([53]),
 tensor([42]),
 tensor([5]),
 tensor([46]),
 tensor([54]),
 tensor([20]),
 tensor([11]),
 tensor([46]),
 tensor([7])]

In [44]:
tmpsmnv = torch.softmax(nv / temp, dim=0)
print(tmpsmnv, tmpsmnv[10], tmpsmnv[20])

tensor([1.0013e-03, 5.6245e-03, 3.0944e-02, 1.5153e-03, 2.4035e-03, 6.6546e-04,
        5.1000e-04, 1.0961e-03, 2.8699e-02, 3.1791e-02, 8.2317e-02, 6.0752e-02,
        9.2664e-04, 4.4433e-02, 3.0766e-03, 1.0716e-02, 3.4920e-02, 2.2607e-02,
        1.0800e-02, 2.8867e-02, 3.1878e-01, 5.2052e-03, 6.0845e-04, 2.9224e-02,
        1.4933e-02, 3.3351e-04, 2.1149e-03, 8.4267e-04, 4.8908e-03, 6.2132e-04,
        2.7892e-04, 9.8023e-03, 1.1317e-03, 1.0131e-03, 4.1035e-02, 2.4511e-02,
        4.5595e-04, 8.6760e-04, 2.7872e-04, 3.1720e-04, 6.6978e-04, 1.1639e-02,
        5.6606e-04, 2.1174e-03, 2.7675e-02, 2.9660e-02, 2.0576e-02, 3.6596e-03,
        4.4671e-04, 3.8402e-04, 1.6254e-03, 3.0115e-04, 1.0277e-02, 1.5137e-03,
        2.9724e-04, 4.5166e-03, 8.2578e-04, 2.2340e-02]) tensor(0.0823) tensor(0.3188)


In [45]:
[torch.multinomial(tmpsmnv, 1) for _ in range(10)]

[tensor([20]),
 tensor([20]),
 tensor([35]),
 tensor([20]),
 tensor([10]),
 tensor([20]),
 tensor([34]),
 tensor([20]),
 tensor([29]),
 tensor([10])]

In [47]:
tmpsmnv[torch.BoolTensor([False, True] + [False for _ in range(56)])]

tensor([0.0056])

In [9]:
import torch
x = torch.rand(3, 5)
print(torch.Tensor([1]).unsqueeze(1), torch.Tensor([1]).unsqueeze(1).shape)
print(x)
print(x / torch.Tensor([1]).unsqueeze(1))

tensor([[1.]]) torch.Size([1, 1])
tensor([[0.1768, 0.2786, 0.2713, 0.6060, 0.6584],
        [0.6335, 0.2532, 0.1173, 0.9449, 0.6471],
        [0.5164, 0.2086, 0.8328, 0.6933, 0.7358]])
tensor([[0.1768, 0.2786, 0.2713, 0.6060, 0.6584],
        [0.6335, 0.2532, 0.1173, 0.9449, 0.6471],
        [0.5164, 0.2086, 0.8328, 0.6933, 0.7358]])


In [5]:
from dataclasses import dataclass, fields
from typing import Callable, Iterable, Iterator, Optional, Tuple, List, Any, Sequence

@dataclass
class Test:
    a: int
    b: float
    
    def __iter__(self) -> Iterator:
        return iter([
            getattr(self, field.name)
            for field in fields(self)
        ])
    

ts = [Test(i, i/2) for i in range(5)]
print(ts)

tl = Test(*map(list, zip(*ts)))
print(tl)

[Test(a=0, b=0.0), Test(a=1, b=0.5), Test(a=2, b=1.0), Test(a=3, b=1.5), Test(a=4, b=2.0)]
Test(a=[0, 1, 2, 3, 4], b=[0.0, 0.5, 1.0, 1.5, 2.0])


In [6]:
import pickle
import torch

d = (58, {'key': 'value'})
s = pickle.dumps(d)
x = torch.frombuffer(s, dtype=torch.int8)
with open('tmp.tmp', 'wb') as f:
    pickle.dump(d, f)